<a href="https://colab.research.google.com/github/Isamaoz/CE888---Data-Science-and-Decision-Making/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv")
df

,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24977,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24978,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24979,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00

In [0]:
arr = df.values[:,1:]
print(arr.shape)

(24982, 100)


In [0]:
import numpy as np

rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810381 (1810381,)


In [0]:
def replace(orig, percentage=0.1):
  new_data=orig.copy()
  rated=np.where(arr!=99)
  n_rated=len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

Use latent factor modeling to infer the hidden ratings of the users (they are labeled as "99" in the dataset) on the training set

In [0]:
n_latent_factors = 2
user_ratings = arr
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1],n_latent_factors))

In [0]:
latent_item_features

array([[0.80650791, 0.30671965],
       [0.39465733, 0.07518567],
       [0.06344226, 0.71418025],
       [0.52517463, 0.49391266],
       [0.05940524, 0.02395212],
       [0.81698108, 0.07606814],
       [0.63329316, 0.7841363 ],
       [0.14126055, 0.18906338],
       [0.91555901, 0.55066647],
       [0.52262587, 0.66672027],
       [0.51294583, 0.18300539],
       [0.87845703, 0.93196389],
       [0.83507177, 0.54654248],
       [0.34371963, 0.11667757],
       [0.33132894, 0.50873757],
       [0.35297331, 0.04682752],
       [0.5540822 , 0.57532237],
       [0.30738375, 0.96604025],
       [0.37267934, 0.60834512],
       [0.0754784 , 0.047842  ],
       [0.23724631, 0.27378365],
       [0.66113907, 0.47660173],
       [0.55369379, 0.49768533],
       [0.66458974, 0.92833992],
       [0.14545239, 0.46575593],
       [0.31078575, 0.57930102],
       [0.87523985, 0.82523707],
       [0.47594042, 0.82154504],
       [0.37830938, 0.44195394],
       [0.70663047, 0.88229842],
       [0.

In [0]:
def predict_rating(user_id,item_id):
  """ Predict a rating given a user_id and an item_id.
  """
  user_preference = latent_user_preferences[user_id]
  item_preference = latent_item_features[item_id]
  return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating !=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        if (iteration % 10) == 0:
            print(mse)

In [0]:
sgd()

26.167621123035467
23.05065340584005
21.18941067458478
19.16999048935224
17.4883503362145
17.22087440764025
17.157613966977898
17.138429339096295
17.132410288112112
17.13070851904686


In [0]:
predictions= latent_user_preferences.dot(latent_item_features.T)
predictions

array([[ 2.98646949,  3.15502096,  3.78912489, ...,  2.51303936,
         3.95447498,  2.75645989],
       [ 6.88241287,  7.18125103,  7.48438442, ...,  7.3358152 ,
         7.2279959 ,  8.26421728],
       [ 1.88386241,  1.93548207,  1.62833711, ...,  2.52819366,
         1.34344296,  2.90608209],
       ...,
       [-0.97053712, -1.03224067, -1.32786878, ..., -0.6972564 ,
        -1.43089534, -0.74795042],
       [-1.91743421, -2.05858914, -2.89149897, ..., -1.04568077,
        -3.23200804, -1.06688507],
       [ 2.78611234,  2.8704649 ,  2.51976696, ...,  3.60095296,
         2.15541885,  4.1269747 ]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 2.986469487025798)","(-0.29, 3.1550209603519415)","(6.36, 3.7891248898194423)","(4.37, 5.265561518364307)","(-2.38, 2.216985600518156)","(-9.66, 2.5206853880429705)","(-0.73, 2.978243153068191)","(-5.34, 2.33222237005769)","(8.88, 5.070904019129473)","(9.22, 2.995690179050072)","(6.75, 2.395888505570721)","(8.64, 2.567544787157425)","(4.42, 3.14722318411351)","(7.43, 2.487270381376009)","(4.56, 3.020886581252098)","(-0.97, 4.318972972410166)","(4.66, 2.4546506635783296)","(-0.68, 2.9524407470942693)","(3.3, 2.728749033840834)","(-1.21, 3.582109295217592)","(0.87, 1.6340981840880129)","(8.64, 3.4919285052279405)","(8.35, 4.017215897824325)","(9.17, 5.510677923360464)","(0.05, 3.540620153819585)","(7.57, 2.434229184488118)","(4.71, 0.24957497145444557)","(0.87, 2.3996124250403965)","(-0.39, 0.1361465907497168)","(6.99, 4.866948829262784)","(6.5, 1.259339356361138)","(-0.92, 0.3379881863693008)","(7.14, 5.210994742250935)","(9.03, 2.776521299597311)","(-1.8, 0.5380290224944231)","(0.73, 0.06116666880288129)","(7.09, 4.85410223349278)","(3.4, 3.0710499857811113)","(-0.87, 2.7942063908206216)","(7.91, 3.2225153597369784)",...,"(-6.7, 1.314145351824907)","(-3.35, 0.8953872471771951)","(-9.03, 3.789443588062977)","(4.47, 4.718676303885914)","(4.08, 1.1645404707636604)","(-3.83, 1.082735931074057)","(8.74, 4.951007217520877)","(1.12, 0.9575512995070344)","(0.78, 1.024444960599817)","(7.52, 3.5386269475440635)","(-5.0, 3.9717242217407818)","(2.77, 1.3073873380576775)","(8.3, 3.102854212300129)","(7.77, 4.954701369327373)","(7.33, 3.786950936173192)","(6.21, 1.4481875160034903)","(7.72, 3.3208810293041564)","(8.98, 2.719302619416563)","(8.64, 3.7808294855198223)","(8.2, 2.8769795258108397)","(3.93, 2.2349828769836626)","(4.85, 3.5186803312245423)","(4.85, 1.8524647037730202)","(6.07, 3.576776013876958)","(8.98, 3.4025236925307363)","(4.51, 3.8814460000233404)","(-0.05, 2.0632558226081077)","(3.69, 2.1539675204771984)","(4.56, 0.4142507942609036)","(0.58, 3.785238733956696)","(2.82, 2.2147330243762853)","(-4.95, 2.920939287340735)","(-0.29, 1.602398992000415)","(7.86, 3.4765989736922642)","(-0.19, 3.0669039485410603)","(-2.14, 2.7723159596460993)","(3.06, 2.419275224969832)","(0.34, 2.5130393639442166)","(-4.32, 3.954474976500863)","(1.07, 2.756459889676717)"
1,"(99.0, 6.88241286629014)","(99.0, 7.181251030322372)","(99.0, 7.484384420503131)","(99.0, 7.252266247849987)","(9.03, 5.514749364126606)","(9.27, 8.64117098613964)","(9.03, 4.968556344698025)","(9.27, 3.023432140187398)","(99.0, 7.62948013095833)","(99.0, 7.265473795677347)","(7.33, 8.353937060145379)","(7.57, 8.920180867221378)","(9.37, 2.870572006008922)","(6.17, 8.847020937654657)","(-6.36, 2.1591321909960257)","(-6.89, 1.91652921536757)","(-7.86, 3.377118066582326)","(9.03, 4.16277228026148)","(9.03, 5.820084425707737)","(9.03, 5.62254006065258)","(7.28, 8.472173196033298)","(99.0, 8.538418672368413)","(8.25, 7.721316362919685)","(99.0, 6.533672925926032)","(99.0, 8.156406993988202)","(7.48, 8.593729442137295)","(7.28, 7.479088416058343)","(7.28, 8.21935512278981)","(8.93, 7.05408825490009)","(99.0, 8.36017686128737)","(6.17, 7.993018856304778)","(7.28, 7.543214154676633)","(99.0, 7.203021260522764)","(99.0, 8.135155796229087)","(8.98, 7.773797580116132)","(7.33, 7.05095500665248)","(99.0, 6.069508698951911)","(6.17, 8.103190584592028)","(9.08, 8.568839274643604)","(7.33, 8.43997675369495)",...,"(6.46, 8.424065673719726)","(7.28, 8.098587771566772)","(99.0, 7.923794098486306)","(99.0, 6.922993776052862)","(7.04, 7.96737474144881)","(7.28, 7.916947535295257)","(99.0, 7.138774932067622)","(7.28, 7.716265012449087)","(8.25, 8.074193952563418)","(99.0, 8.096654100112243)","(99.0, 6.218409158333439)","(99.0, 8.302239428826253)","(99.0, 8.42

In [0]:
#Changing the number of latent factors while all the other hyper-parameters stay the same
n_latent_factors=4
user_ratings=arr
latent_user_preferences=np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features=np.random.random((user_ratings.shape[1],n_latent_factors))
sgd()
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data

25.673617528089522
23.720534257748465
19.48525528569798
17.584819281285277
17.323060939533477
17.163421123465245
16.95401802031274
16.656602419403338
16.35735688851567
16.09220818445186


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,"(4.08, 1.781212118139185)","(-0.29, 3.026128755521662)","(6.36, 3.5002445031172305)","(4.37, 6.101765068649042)","(-2.38, 2.1840675142336368)","(-9.66, 3.070456878540188)","(-0.73, 1.7429801700386487)","(-5.34, 0.8018093136007439)","(8.88, 4.107341366828446)","(9.22, 1.5954920259528695)","(6.75, 2.4380412126062225)","(8.64, 3.1426848637608797)","(4.42, 2.479581379388762)","(7.43, 3.3682684515178583)","(4.56, 1.3790543796027186)","(-0.97, 3.1452715154050876)","(4.66, 3.15339222598005)","(-0.68, 1.0820223589331883)","(3.3, 2.1403371017564266)","(-1.21, 3.4598452906045254)","(0.87, 2.519323916736077)","(8.64, 3.4292920806943723)","(8.35, 3.32719783183436)","(9.17, 5.2507860935356)","(0.05, 3.853392406325511)","(7.57, 3.283675164701971)","(4.71, 0.17249290854171154)","(0.87, 2.1344957951977794)","(-0.39, -0.46451204972345606)","(6.99, 5.241233053172096)","(6.5, 1.837677165112864)","(-0.92, -0.3781703197580393)","(7.14, 5.341081776085081)","(9.03, 3.5241658897518886)","(-1.8, -0.08174712644817182)","(0.73, -0.6859299005942139)","(7.09, 3.7914422426638996)","(3.4, 1.963664674758413)","(-0.87, 3.8845395097317432)","(7.91, 3.3472685116976937)",...,"(-6.7, 1.542088787449219)","(-3.35, 0.07600909625734116)","(-9.03, 3.2447024790663277)","(4.47, 3.661918829948148)","(4.08, 1.8299540945396389)","(-3.83, 0.7183765679341594)","(8.74, 4.543691055898195)","(1.12, 0.48859727339925985)","(0.78, 0.8221411902388941)","(7.52, 3.88527530854674)","(-5.0, 4.398989289871372)","(2.77, 1.8329065173190267)","(8.3, 4.670000885897919)","(7.77, 5.807266740540043)","(7.33, 5.044486423675851)","(6.21, 1.9439227411241486)","(7.72, 4.479177687251969)","(8.98, 3.6860491179131993)","(8.64, 5.337995105480538)","(8.2, 4.52700747312822)","(3.93, 2.61129430672248)","(4.85, 3.5277296793322748)","(4.85, 2.839089771218508)","(6.07, 3.6471259176392627)","(8.98, 4.336298403974382)","(4.51, 3.0037685889914636)","(-0.05, 2.107630951259358)","(3.69, 1.6347617390226226)","(4.56, 0.018922230655823356)","(0.58, 3.8625999053544975)","(2.82, 1.6784314443631567)","(-4.95, 3.143380739953757)","(-0.29, 1.4373749033939824)","(7.86, 2.566763964368854)","(-0.19, 2.648844865347045)","(-2.14, 2.4421392686781886)","(3.06, 1.7831652454645117)","(0.34, 3.9247260786580704)","(-4.32, 4.260487564005898)","(1.07, 4.089104399722315)"
1,"(99.0, 8.419020439874066)","(99.0, 7.2724436644227115)","(99.0, 7.378312672687073)","(99.0, 4.7598132751538484)","(9.03, 5.831838076348032)","(9.27, 7.484818570107039)","(9.03, 6.623439091336774)","(9.27, 5.081456498008538)","(99.0, 7.859531647838871)","(99.0, 9.240241406842001)","(7.33, 8.30142997568589)","(7.57, 8.020550799911629)","(9.37, 3.6531551539315377)","(6.17, 7.503991823986079)","(-6.36, 4.382815854392809)","(-6.89, 2.7218562832950437)","(-7.86, 2.0898728877372563)","(9.03, 6.64600449656128)","(9.03, 6.295142147647234)","(9.03, 5.1394925557995474)","(7.28, 7.567370723913711)","(99.0, 8.382595450569687)","(8.25, 7.935698808367166)","(99.0, 5.671239563632589)","(99.0, 7.428483076289817)","(7.48, 7.398462770351505)","(7.28, 8.302056281857022)","(7.28, 8.239766711475113)","(8.93, 8.797268418999641)","(99.0, 6.697649865686604)","(6.17, 7.638933457953309)","(7.28, 9.054622565286651)","(99.0, 5.912969689386431)","(99.0, 6.981652332911059)","(8.98, 9.284839353434611)","(7.33, 8.875353984595902)","(99.0, 6.67427461735479)","(6.17, 9.532736589564417)","(9.08, 6.754787024827825)","(7.33, 7.967666455761953)",...,"(6.46, 8.37633588210767)","(7.28, 9.497285869655817)","(99.0, 7.776133867841285)","(99.0, 7.60764067342838)","(7.04, 7.398028612625956)","(7.28, 8.78455248438159)","(99.0, 6.536647637029227)","(7.28, 8.451570779724404)","(8.25, 8.785759174722932)","(99.0, 6.867874237156426)","(99.0, 3.9719119639502924)","(99.0, 7.14306559

When the number of latent factors increases
 from 2 to 4, the MSE decreases in most of the cases